# **MA Cross Over S&P 500**

This code is a backtest for optimising the moving averages (MA). The strategy works when the more sensitive moving average (lesser number of days used) crosses above the less sensitive moving average (greater number of days considered), the code opens a long position.

The data used is sourced from yfinance, and is training data set from 01/01/2015, up to just after the first servere drop due to the COVID pandemic (Ending around November 2020). 

This is an optimisation test, where 2 for loops will be used, to test a range of sensitive MAs, with a range of less sensitive MAs. The classical strategy which many retail traders may consider is the 20 day MA with the 50 day MA, but here, we will determine which combination proves best, based on the backtest. Hence no forward testing is considered in this code. 

Each combination, the considerations will include: the number of trades made, average point increase or decrease per trade (points due to the S&P 500 being an index), the total point increase or decrease, the success rate (which is defined by the percentage of winning trades, a winning trade is a trade which has a positive point increase), maximum equity drawdown, maximum equity drawup, sharpe ratio (here the risk free rate for each year is taken as 1.85%). 

## Importing useful libraries

In [5]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
import statistics

## Importing financial data

In [6]:
symbol="SPY"

spy_data=yf.download(symbol, start="2015-01-01", end="2023-05-31")

[*********************100%***********************]  1 of 1 completed


## Split data into a train and test split (70% for test and 30% for training)

In [7]:
train_df, test_df=train_test_split(spy_data, test_size=0.3, shuffle=False)

## Here, this strategy will be working based on the 'Closing' value of the candles 

In [8]:
train_df=train_df.iloc[:,3]
test_df=test_df.iloc[:,3]

## Defining functions

2 moving averages are considered, for example a 20 day MA with a 40 MA. To represent a time to open a long position, the 20 day MA must cross over the 40 day MA. Therefore we need the 2 MA values for our current time period (today), and the previous time period (yesterday). With these 4 values, a cross over can be indentified, with signals an oppurtunity to open a long position.

In [9]:
def MA_ONE_today(train_df, MA_one, i):
    ma_one_today=train_df.iloc[i-MA_one+1:i+1].mean()
    return ma_one_today
def MA_TWO_today(train_df, MA_two, i):
    ma_two_today=train_df.iloc[i-MA_two+1:i+1].mean()
    return ma_two_today
def MA_ONE_yesterday(train_df, MA_one, i):
    ma_one_yesterday=train_df.iloc[i-MA_one:i].mean()
    return ma_one_yesterday
def MA_TWO_yesterday(train_df, MA_two, i):
    ma_two_yesterday=train_df.iloc[i-MA_two:i].mean()
    return ma_two_yesterday
def Sharpe_ratio_formula(year_expected_returns, Risk_free_rate):
    average_expected_return=pd.Series(year_expected_returns).mean()
    std_dev=statistics.stdev(year_expected_returns)
    Sharpe_ratio=(average_expected_return-Risk_free_rate)/std_dev
    return Sharpe_ratio

## Defining empty lists, to append results to, and the risk free rate of 1.85%

In [16]:
first_moving_average=[]
second_moving_average=[]
average_point=[]
numb_trades=[]
Total_point=[]
Success_rate=[]
max_equity_drawdown=[]
max_equity_drawup=[]
Sharpe_ratios=[]
Risk_free_rate=1.85

## Now the Main body loop

The first for loop defines the sensitive MA. The second for loop defines the less sensitive MA. 

Then, for each combination the variables must be re-definded: trade_counter (number of trades made for this given combination) must be reset to 0, the total points made from this strategy must be reset to zero (points=0), etc etc. A list is defined year_expected_returns, this calculates the percentage increased, based on points increased (or decreased) over points required to invest. This is key, when calculating our sharpe ratio. 

Then the next for loop ranges historically through our training data set, where various combinations will now be tested.

The functions are called, so the MAs for today are calculated, and the MAs for yesterday are calculated (a single row represents a single day). 

**BUYING CONDITIONS**
To represent a buying condition (a MA cross over), the sensitive MA of today, must be greater then the less sensitive MA today (ma_one_today>ma_two_today), and the sensitive MA of yesterday must be less than the less sensitive MA of yesterday (ma_one_yesterday<ma_two_yesterday). And we must not currently have an open condition, hence open_position==0.

With this condition satisfied, our buy price is now the current price: buy_price=train_df.iloc[i]

This model therefore assumes zero slippage and ultra-low latency.

**SELLING CONDITIONS**
The only selling condition is based on when the sensitive MA crosses back below the less sensitive MA. And we have an current long position open.

elif ma_one_today<ma_two_today and ma_one_yesterday>ma_two_yesterday and open_position==1:

If this condition is met, the selling of our long postion is determined: sell_price=train_df.iloc[i]

Therefore it can now be calculated if our trade has made an increase in points, or decrease, this is stored as the variable profit_loss. profit_loss=sell_price-buy_price.

The equity drawdown or drawup of each trade, for each combination is stored in equity_drawdowns_drawups, and at the end of the loop for that specific combination, the max value (maximum equity drawup) and min value (maximum equity drawdown) will be taken. 

**Further code**
Information for the sharpe ratio calculation is then calculated, when the code has iterated through 252 days (assuming 252 trading days per year in the US, this counter will then be set to zero), however the expected annual return for that year will be calcualted and append to a list, which will later be converted to a Series. 

Here is the main body:


In [35]:
for MA_one in range(5,49):
    
    for MA_two in range(15,120):
        
        open_position=0
        trade_counter=0
        points=0
        point_increase_start_of_this_year=0
        point_increase_this_year=0
        winning_trades=0
        second_counter=0
        equity_drawdowns_drawups=[]
        year_expected_returns=[]
        

        for i in range(MA_two,len(train_df)-1):
            ma_one_today=MA_ONE_today(train_df, MA_one, i)
            ma_two_today=MA_TWO_today(train_df, MA_two, i)
            ma_one_yesterday=MA_ONE_yesterday(train_df, MA_one, i)
            ma_two_yesterday=MA_TWO_yesterday(train_df, MA_two, i)
            second_counter+=1
            
            
            if (ma_one_today>ma_two_today and
            ma_one_yesterday<ma_two_yesterday and 
            open_position==0):
                buy_price=train_df.iloc[i]
                
                open_position=1
            elif (ma_one_today<ma_two_today and 
            ma_one_yesterday>ma_two_yesterday and 
            open_position==1):
                sell_price=train_df.iloc[i]
                
                profit_loss=sell_price-buy_price
                percentage_win_loss=(sell_price-buy_price)/buy_price*100
                equity_drawdowns_drawups.append(percentage_win_loss)
                
                points=points+profit_loss
                
                open_position=0
                
                trade_counter+=1
                
                if profit_loss>0:
                    winning_trades+=1
                    
            if second_counter==252:
                
                point_increase_this_year=points-point_increase_start_of_this_year
                point_increase_start_of_this_year=points
                mean_cost=train_df.iloc[i-252+1:i+1].mean()
                expected_return=point_increase_this_year/mean_cost*100
                year_expected_returns.append(expected_return)
                second_counter=0
                
        
                
                        
        if trade_counter==0:
            win_rate=0
            greatest_equity_drawdown=0
            greatest_equity_drawup=0
            Sharpe_ratio=0
            average_pnt=points
        else:
            win_rate=winning_trades/trade_counter
            average_pnt=points/trade_counter
            greatest_equity_drawdown=min(equity_drawdowns_drawups)
            greatest_equity_drawup=max(equity_drawdowns_drawups)
            Sharpe_ratio=Sharpe_ratio_formula(year_expected_returns, Risk_free_rate)
            
            
        numb_trades.append(trade_counter)
        first_moving_average.append(MA_one)
        second_moving_average.append(MA_two)
        average_point.append(average_pnt)
        Success_rate.append(win_rate)
        max_equity_drawdown.append(greatest_equity_drawdown)
        max_equity_drawup.append(greatest_equity_drawup)
        Sharpe_ratios.append(Sharpe_ratio)
        Total_point.append(points)

In [36]:
df_results=pd.DataFrame({'First Moving Average': first_moving_average,
                         'Second Moving Average': second_moving_average,
                         'Revenue for these MAs':average_point, 
                         'Number of trades': numb_trades, 
                         'Total Point Increase/Decrease': Total_point, 
                         'Success Rate': Success_rate,
                         'Maximum equity drawdown (Percentage)': max_equity_drawdown, 
                         'Maximum equity drawup (Percentage)': max_equity_drawup, 
                         'Sharpe Ratio': Sharpe_ratios })            

In [37]:
df_results = df_results.sort_values(by='Sharpe Ratio', ascending=False)
print(df_results)

      First Moving Average  Second Moving Average  Revenue for these MAs  \
6021                    18                     23               2.785430   
1373                    18                     23               2.785430   
2219                    26                     29               3.497418   
6867                    26                     29               3.497418   
5710                    15                     27               2.267776   
...                    ...                    ...                    ...   
4329                    46                     39               1.262353   
4121                    44                     41               1.095833   
8769                    44                     41               1.095833   
9082                    47                     39               1.660623   
4434                    47                     39               1.660623   

      Number of trades  Total Point Increase/Decrease  Success Rate  \
6021            

From this information, the combination of using the more sensitive MA (=18) and the less sensitive MA (=23) results in a sharpe ratio of 2.76. The maximum equity drawdown is 10.9% of the accounts portfolio (assuming the total cash value has been invested). The total point increase over this trading period is 97.5 points. This overall increase does not compare to a traditional buy and hold of the S&P 500 which equates to the following value (+155.19):


In [38]:
Trad_buy_and_hold = train_df.iloc[-1] - train_df.iloc[0]
print("Overall point increase or decrease over this testing period is:",
      Trad_buy_and_hold)

Overall point increase or decrease over this testing period is: 155.19000244140625
